In [ ]:
# Install required packages if not already installed
import sys
import subprocess

def install_if_missing(package):
    try:
        __import__(package)
        print(f"✅ {package} is already installed")
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ {package} has been installed")

# Install necessary packages
install_if_missing('voila')
install_if_missing('ipywidgets')
install_if_missing('psycopg2-binary')
install_if_missing('pandas')
install_if_missing('matplotlib')
install_if_missing('sqlalchemy')
install_if_missing('python-dotenv')

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from sqlalchemy import create_engine, text
import os
from datetime import datetime, timedelta
import warnings
from dotenv import load_dotenv
from pathlib import Path

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set plot styling
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)

# Create a custom style for the dashboard
display(HTML("""
<style>
.dashboard-title {
    color: #2F5496;
    font-size: 24px;
    font-weight: bold;
    padding: 10px 0;
}
.section-title {
    color: #2F5496;
    font-size: 18px;
    font-weight: bold;
    padding: 5px 0;
}
.info-text {
    color: #444;
    font-size: 14px;
    padding: 5px 0;
}
.data-box {
    background-color: #f0f0f0;
    border-radius: 5px;
    padding: 10px;
    margin: 10px 0;
}
</style>
"""))

## Database Connection

First, let's establish a connection to the TimescaleDB database where our MPP data is stored.

In [ ]:
# Load environment variables from .env file if available
dotenv_path = Path("../.env")
if dotenv_path.exists():
    load_dotenv(dotenv_path)
    print(f"Loaded environment variables from {dotenv_path}")

# Database configuration from environment variables with fallbacks
DB_CONFIG = {
    'host': os.getenv('POSTGRES_HOST', 'timescaledb'),
    'port': int(os.getenv('POSTGRES_PORT', 5432)),
    'database': os.getenv('POSTGRES_DB', 'perocube'),
    'user': os.getenv('POSTGRES_USER', 'postgres'),
    'password': os.getenv('POSTGRES_PASSWORD', 'postgres')
}

# Create SQLAlchemy engine with connection string
connection_string = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
engine = create_engine(connection_string)

# Helper function to run SQL queries
def run_query(query, conn=None):
    """Execute SQL query and return results as DataFrame"""
    close_conn = False
    if conn is None:
        conn = engine.connect()
        close_conn = True
    
    try:
        result = pd.read_sql_query(query, conn)
        return result
    except Exception as e:
        print(f"Error executing query: {e}")
        print(f"Query: {query}")
        return None
    finally:
        if close_conn and conn:
            conn.close()

# Test connection and check if our functions exist
try:
    with engine.connect() as conn:
        result = conn.execute(text("SELECT version();"))
        version = result.scalar()
        print(f"✅ Successfully connected to the database.")
        
        # Check if our SQL functions are available
        result = conn.execute(text("""
            SELECT routine_name 
            FROM information_schema.routines 
            WHERE routine_name IN ('get_queryable_solar_cell_pixels', 'get_pixel_activity_range', 'get_mpp_data_for_pixel')
            AND routine_type = 'FUNCTION'
        """))
        
        functions = [row[0] for row in result]
        required_funcs = ['get_queryable_solar_cell_pixels', 'get_pixel_activity_range', 'get_mpp_data_for_pixel']
        
        if all(func in functions for func in required_funcs):
            print(f"✅ All required MPP functions are available.")
        else:
            missing = [func for func in required_funcs if func not in functions]
            print(f"⚠️ Warning: Some MPP functions are missing: {', '.join(missing)}")
            print(f"Available functions: {', '.join(functions) if functions else 'None'}")
            
except Exception as e:
    print(f"❌ Error connecting to database: {e}")

## Interactive Dashboard

Now let's create an interactive dashboard that allows users to select a solar cell pixel and analyze its MPP data.

In [ ]:
# Get available solar cells and pixels
def get_available_pixels():
    """Get all available solar cells and pixels that have data"""
    # First get all queryable pixels
    query = """SELECT * FROM get_queryable_solar_cell_pixels();"""
    queryable_pixels = run_query(query)
    
    if queryable_pixels is None or queryable_pixels.empty:
        return None
    
    # Format for dropdown selection
    pixels_options = []
    for _, row in queryable_pixels.iterrows():
        cell = row['solar_cell_name']
        pixel = row['pixel_identifier']
        first_conn = row['first_connection_event_time']
        last_event = row['last_event_time']
        pixels_options.append(
            (f"{cell} - {pixel} ({first_conn.strftime('%Y-%m-%d')} to {last_event.strftime('%Y-%m-%d')})", 
             (cell, pixel))
        )
    
    return pixels_options

# Function to check if a pixel has MPP data
def check_pixel_has_data(solar_cell, pixel):
    """Check if a pixel has MPP data and return count"""
    # Use the overloaded function without explicit time range
    query = f"""SELECT COUNT(*) as count FROM get_mpp_data_for_pixel('{solar_cell}', '{pixel}');"""
    result = run_query(query)
    count = result['count'].iloc[0] if result is not None and not result.empty else 0
    return count > 0, count

def get_pixel_activity_range(solar_cell, pixel):
    """Get the activity range for a pixel"""
    query = f"""
    SELECT * FROM get_pixel_activity_range('{solar_cell}', '{pixel}');
    """
    result = run_query(query)
    if result is not None and not result.empty:
        return result['calculated_start_datetime'].iloc[0], result['calculated_end_datetime'].iloc[0]
    return None, None

# Function to get data and create plots
def get_mpp_data(solar_cell, pixel, start_time=None, end_time=None):
    """Get MPP data for a pixel"""
    
    # If start and end time are provided, use them, otherwise use the default range
    if start_time is not None and end_time is not None:
        query = f"""
        SELECT * FROM get_mpp_data_for_pixel(
            '{solar_cell}', 
            '{pixel}', 
            '{start_time}'::TIMESTAMP WITH TIME ZONE, 
            '{end_time}'::TIMESTAMP WITH TIME ZONE
        );
        """
    else:
        query = f"""
        SELECT * FROM get_mpp_data_for_pixel('{solar_cell}', '{pixel}');
        """
    
    mpp_data = run_query(query)
    return mpp_data

# Function to create plots
def create_plots(mpp_data, pixel_info):
    """Create the visualization plots for the dashboard"""
    
    if mpp_data is None or mpp_data.empty:
        return []
    
    solar_cell, pixel = pixel_info
    pixel_to_analyze = f"{solar_cell}-{pixel}"
    
    # Time series plot of power
    fig_power = plt.figure(figsize=(12, 5))
    plt.plot(mpp_data['timestamp'], mpp_data['power'], 'b-', alpha=0.6)
    plt.title(f'MPP Power Output for {pixel_to_analyze}')
    plt.xlabel('Time')
    plt.ylabel('Power (W)')
    plt.grid(True)
    plt.tight_layout()
    
    # IV scatter plot
    fig_iv = plt.figure(figsize=(10, 5))
    plt.scatter(mpp_data['voltage'], mpp_data['current'], alpha=0.5, s=10)
    plt.title(f'Current vs. Voltage for {pixel_to_analyze}')
    plt.xlabel('Voltage (V)')
    plt.ylabel('Current (A)')
    plt.grid(True)
    plt.tight_layout()
    
    # Power histogram
    fig_hist = plt.figure(figsize=(10, 5))
    plt.hist(mpp_data['power'], bins=30, alpha=0.7)
    plt.title(f'Distribution of Power Values for {pixel_to_analyze}')
    plt.xlabel('Power (W)')
    plt.ylabel('Frequency')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    
    return [fig_power, fig_iv, fig_hist]

In [ ]:
# Main dashboard function
def run_dashboard():
    """Create and run the interactive dashboard"""
    
    # Get available pixels
    pixels_options = get_available_pixels()
    
    if not pixels_options:
        display(HTML("<div class='info-text'>No queryable pixels found in the database.</div>"))
        return
    
    # Create widgets
    pixel_dropdown = widgets.Dropdown(
        options=pixels_options,
        description='Pixel:',
        disabled=False,
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='80%')
    )
    
    # Time range widgets (initially disabled)
    use_custom_time_range = widgets.Checkbox(
        value=False,
        description='Use custom time range',
        disabled=False
    )
    
    # Use flexible layout for the datetime pickers
    start_date_picker = widgets.DatePicker(
        description='Start:',
        disabled=True,
        layout=widgets.Layout(width='auto')
    )
    
    end_date_picker = widgets.DatePicker(
        description='End:',
        disabled=True,
        layout=widgets.Layout(width='auto')
    )
    
    # View button
    view_button = widgets.Button(
        description='View Data',
        disabled=False,
        button_style='primary',
        icon='chart-line'
    )
    
    # Output area for plots
    output_area = widgets.Output()
    
    # Status area
    status_area = widgets.Output()
    
    # Function to update time range widgets when a new pixel is selected
    def on_pixel_change(change):
        with status_area:
            clear_output()
            if change['new']:
                solar_cell, pixel = change['new']
                print(f"Checking data for {solar_cell} - {pixel}...")
                
                has_data, count = check_pixel_has_data(solar_cell, pixel)
                if has_data:
                    start_time, end_time = get_pixel_activity_range(solar_cell, pixel)
                    if start_time and end_time:
                        start_date_picker.value = start_time.date()
                        end_date_picker.value = end_time.date()
                        print(f"Found {count} data points from {start_time.date()} to {end_time.date()}")
                    else:
                        print(f"Found {count} data points but couldn't determine time range")
                else:
                    print("No data available for this pixel")
    
    # Function to toggle time range widgets
    def on_custom_time_toggle(change):
        start_date_picker.disabled = not change['new']
        end_date_picker.disabled = not change['new']
    
    # Function to view the data and create plots
    def on_view_button_click(b):
        with output_area:
            clear_output()
            solar_cell, pixel = pixel_dropdown.value
            
            with status_area:
                clear_output()
                print(f"Loading data for {solar_cell} - {pixel}...")
            
            # Get time range (either custom or default)
            if use_custom_time_range.value and start_date_picker.value and end_date_picker.value:
                start_time = datetime.combine(start_date_picker.value, datetime.min.time())
                end_time = datetime.combine(end_date_picker.value, datetime.max.time())
                mpp_data = get_mpp_data(solar_cell, pixel, start_time, end_time)
            else:
                mpp_data = get_mpp_data(solar_cell, pixel)
            
            if mpp_data is None or mpp_data.empty:
                with status_area:
                    clear_output()
                    print(f"No data found for {solar_cell} - {pixel} in the specified time range")
                return
            
            # Display data summary
            data_start_time = mpp_data['timestamp'].min()
            data_end_time = mpp_data['timestamp'].max()
            duration = data_end_time - data_start_time
            
            display(HTML(f"""
            <div class='dashboard-title'>MPP Data Analysis for {solar_cell} - {pixel}</div>
            <div class='data-box'>
                <div class='section-title'>Data Summary</div>
                <div class='info-text'>Time Range: {data_start_time} to {data_end_time}</div>
                <div class='info-text'>Duration: {duration}</div>
                <div class='info-text'>Number of Data Points: {len(mpp_data):,}</div>
                <div class='info-text'>Average Power: {mpp_data['power'].mean():.4f} W</div>
                <div class='info-text'>Max Power: {mpp_data['power'].max():.4f} W</div>
            </div>
            """))
            
            # Create and display plots
            figs = create_plots(mpp_data, (solar_cell, pixel))
            if figs:
                for fig in figs:
                    display(fig)
            
            with status_area:
                clear_output()
                print(f"✅ Successfully loaded {len(mpp_data):,} data points")
    
    # Connect event handlers
    pixel_dropdown.observe(on_pixel_change, names='value')
    use_custom_time_range.observe(on_custom_time_toggle, names='value')
    view_button.on_click(on_view_button_click)
    
    # Create layout
    header = widgets.HTML("<div class='dashboard-title'>MPP Data Analysis Dashboard</div>")
    
    controls = widgets.VBox([
        pixel_dropdown,
        widgets.HBox([use_custom_time_range]),
        widgets.HBox([start_date_picker, end_date_picker]),
        view_button,
        status_area
    ])
    
    dashboard = widgets.VBox([
        header,
        controls,
        output_area
    ])
    
    display(dashboard)

In [ ]:
# Run the dashboard
run_dashboard()